<a href="https://colab.research.google.com/github/AnIsAsPe/RNN-Resumenes-de-Texto/blob/main/Word_embeddings_Glove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vectores de palabras (word embeddings), de [GloVe](https://nlp.stanford.edu/projects/glove/), previamente entrenadas

Recursos:

[Embedding Projector](https://projector.tensorflow.org/)


In [ ]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!unzip -q glove.6B.zip

--2021-02-18 18:41:55--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-02-18 18:41:55--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-02-18 18:41:55--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:

dic_vectores_glove  = {}
with open('/content/drive/MyDrive/Datos/glove.6B.100d.txt') as f:
    for line in f:
        palabra, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        dic_vectores_glove [palabra] = coefs

#Cantidad de palabras en glove
len(dic_vectores_glove )

400000

In [ ]:
glove_df = pd.DataFrame(dic_vectores_glove)
glove_df.shape

(100, 400000)

In [64]:
glove_df = glove_df.transpose(copy=True)
glove_df.shape

(400000, 100)

In [80]:
palabras = 'boy girl husband wife man woman'.split()
X = glove_df.loc[palabras,:]
X

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
boy,0.89461,0.37758,0.42067,-0.513340,-0.282980,1.00120,0.187480,0.218680,-0.030053,0.086727,-0.44626,0.34755,0.61516,0.31583,0.56543,0.403140,0.45648,-0.39995,-0.20669,0.84975,-0.016242,-0.281520,-0.052972,-0.19438,0.35783,0.89044,-0.92628,-0.88332,0.089802,1.385600,0.118840,0.36608,0.440090,0.516170,0.35074,-0.24373,-0.68049,0.132760,0.78618,-0.525300,...,0.80694,1.01290,-0.518530,-0.211520,0.35337,0.047413,0.75148,0.31402,-0.42139,0.299540,0.32677,0.121790,-0.19638,0.12719,0.041816,1.050000,0.177030,0.08307,-0.51415,-0.652300,0.171470,0.37803,0.110700,-0.255180,-1.07120,-0.51485,-0.381630,-0.423140,0.135030,0.014892,0.11282,0.26315,0.20852,-0.016587,-0.40463,-0.159090,-0.71309,-0.13668,0.040350,-0.84151
girl,0.74582,0.19531,0.46831,-0.102770,-0.299270,0.99653,0.139400,0.641730,0.331290,0.353220,-0.30205,0.34699,0.53364,0.62235,0.43555,0.544030,0.39656,-0.31789,-0.20172,0.99636,0.001848,-0.148260,-0.004184,0.24953,0.30177,1.00430,-0.44867,-1.29110,-0.127410,0.901030,0.118700,0.27229,0.976970,0.019520,0.40453,-0.61585,-0.50483,-0.031174,0.70384,-0.460580,...,0.73902,1.09980,-0.766330,-0.000848,0.50982,0.195870,0.71845,0.25045,-0.29609,0.190050,0.19021,-0.216680,-0.25819,0.45417,0.151850,1.350000,0.322840,-0.23826,-0.69976,-0.981570,-0.393360,0.52666,0.601270,-0.175200,-1.00140,-1.04900,-0.401130,-0.199300,0.034218,-0.170690,0.11801,0.29238,0.44989,-0.211550,-0.51509,0.112150,-0.95016,-0.19681,-0.146210,-0.30505
husband,0.61534,0.27597,0.29280,-0.250640,-0.105560,0.90526,-0.141570,0.164830,0.058663,0.266470,0.14610,0.82343,0.17578,0.49482,0.56132,-0.761280,0.27528,-0.49246,-0.71117,1.10400,-0.260500,-0.092159,0.330570,0.49800,0.35827,0.12743,-0.64898,-0.88145,0.530480,-0.051134,0.319930,0.93357,1.070400,-0.045996,-0.47942,0.48067,0.45560,-0.077096,0.47297,0.126090,...,0.53979,0.52270,0.222680,-0.208500,0.58320,-0.094776,0.95910,0.39932,0.02477,0.688390,-0.44054,0.249640,-0.17609,-0.24149,0.100330,0.386680,-0.167830,-0.35319,-0.50902,-0.084276,0.020392,0.82435,0.803630,0.349550,-0.97259,-0.61299,0.026339,0.322630,-0.417260,-1.366400,0.48827,-0.51077,1.04260,0.084176,-0.30587,0.138590,-0.29958,-0.13959,-0.095047,-0.74174
wife,0.33954,-0.41367,0.22616,-0.040106,0.104890,0.72435,0.092193,0.440020,0.323460,0.136900,-0.14326,0.62633,0.17516,0.74118,0.20934,-0.710180,0.20261,-0.33897,-0.74968,1.07650,-0.408060,0.094819,0.313500,0.86049,0.43735,0.25478,-0.64706,-1.31420,0.396230,0.358770,0.309250,1.07540,1.225000,0.124110,-0.49617,0.37415,0.54215,0.148410,0.66266,0.294810,...,0.17978,1.01760,0.127180,-0.122170,0.34378,-0.240030,1.01470,0.68566,0.19761,0.352220,-0.31113,0.051390,-0.12831,-0.23403,0.365150,0.589900,0.018485,-0.26781,-0.76352,-0.318170,0.459300,0.63011,1.068900,0.254900,-1.18960,-1.08040,-0.085283,-0.022989,-0.158270,-1.135100,0.39241,-0.71972,1.43430,0.297410,-0.60843,0.071802,-0.35413,-0.48860,-0.087106,-0.59801
man,0.37293,0.38503,0.71086,-0.659110,-0.001013,0.92715,0.276150,-0.056203,-0.242940,0.246320,-0.18449,0.31398,0.48983,0.09256,0.32958,0.150560,0.57317,-0.18529,-0.52277,0.46191,0.920380,0.031001,-0.162460,-0.40567,0.78621,0.57722,-0.53501,-0.68228,0.169870,0.363100,-0.071773,0.47233,0.027806,-0.149510,0.17543,-0.37573,-0.78517,0.581710,0.86859,0.031445,...,0.41151,0.84328,-0.542590,-0.167150,0.73927,-0.093477,0.90278,0.50889,-0.50031,0.264510,0.15443,-0.294320,0.10906,-0.26667,0.354380,0.049079,0.180180,-0.58590,-0.55542,-0.289870,0.742780,0.34530,-0.028757,-0.226460,-1.31130,-0.57190,-0.523060,-0.126700,-0.098678,-0.534630,0.28607,-0.37501,0.45742,0.045975,-0.24675,0.045656,-0.38302,-0.93711,0.039138,-0.53911
woman,0.59368,0.44825,0.59320,0.074134,0.111410,1.27930,0.166560,0.240700,0.390450,0.327660,-0.75034,0.35007,0.76057,0.38067,0.17517,0.031791,0.46849,-0.21653,-0.46282,0.39

In [81]:
import plotly.express as px
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
components = pd.DataFrame(pca.fit_transform(X), index = X.index)
components.reset_index(inplace=True)

fig = px.scatter_3d( 
    components, x=0, y=1, z=2, color='index',
    labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
)
fig.show()

In [110]:
def similitud_coseno(a_vector, b_vector):
    '''Calcula la similitud coseno entre los vectores a y b'''

    numerador = np.dot(a_vector, b_vector)
    
    a_norm = np.sqrt(np.sum(a_vector**2))  
    b_norm = np.sqrt(np.sum(b_vector**2))
    
    denominador = a_norm * b_norm
    
    similitud_coseno = numerador / denominador
    
    return similitud_coseno

In [111]:
cat = glove_df.loc['cat']
dog = glove_df.loc['dog']
lion = glove_df.loc['lion']
wolf = glove_df.loc['wolf']

In [112]:
similitud_coseno(cat, dog)

0.8798076

In [113]:
similitud_coseno(lion, dog)

0.49642202

In [114]:
similitud_coseno(wolf, dog)

0.5369378

In [122]:
def encuentra_palabra(a, b, c):
    a, b, c = a.lower(), b.lower(), c.lower()
    a_vector, b_vector, c_vector = dic_vectores_glove[a], dic_vectores_glove[b], dic_vectores_glove[c]
    
    vocabulario = dic_vectores_glove.keys()
    max_similitud = -1000 #la inicializamos con una similaridad alejada
    mas_parecida = None
    
    for palabra in vocabulario:
        if palabra in [a, b, c]:
            continue
            
        sim_coseno = similitud_coseno(np.subtract(b_vector, a_vector), np.subtract(dic_vectores_glove[palabra], c_vector))
        
        if sim_coseno > max_similitud:
            max_similitud = sim_coseno
            mas_parecida = palabra
            
    return mas_parecida, sim_coseno

In [128]:
encuentra_palabra('man', 'woman', 'boy')

('girl', 0.0014003076)

In [136]:
encuentra_palabra('italia', 'spaghetti', 'mexico')

('sauce', -0.005849106)

In [139]:
encuentra_palabra('grass', 'green', 'sky')

('blue', -0.27172074)

In [126]:
encuentra_palabra('spain', 'madrid', 'japan')

('tokyo', 0.2830456)

In [133]:
encuentra_palabra('france', 'french', 'finland')

('finnish', 0.26130632)